In [2]:
import numpy as np

In [9]:
class Loss:
    def __init__(self, loss_type="mse"):
        self.loss_type = loss_type
        self.loss_function = self._get_loss_function()

    def _get_loss_function(self):
        if self.loss_type == "mse":
            return self.mean_squared_error
        elif self.loss_type == "cross_entropy":
            return self.cross_entropy_loss
        # Add more loss functions as needed

    def mean_squared_error(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)

    def cross_entropy_loss(self, y_true, y_pred):
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    # Add more loss functions as needed

    def __call__(self, y_true, y_pred):
        return self.loss_function(y_true, y_pred)

# # Example usage:
loss_instance = Loss(loss_type="cross_entropy")
# sample_y_true = np.array([1, 0, 1, 0])
# sample_y_pred = np.array([1, 0, 1, 0])

# result = loss_instance(sample_y_true, sample_y_pred)
# print(loss_instance.loss_type)
# print(f"Loss: {result}")


cross_entropy
Loss: 9.992007221626415e-16


In [ ]:

class Layer:
    def __init__(self, activation_function='linear', num_neurons=1):
        self.activation = activation_function
        self.activation_function = self._activation_function()
        self.N = num_neurons
        self.prev_layer = None
        self.next_layer = None
        self.input_weights = None  # Matrix of size NxK
        self.output_weights = None  # Matrix of size WxN
    
    def _get_loss_function(self):
        if self.loss_type == "linear":
            return self.linear_activation_function
        elif self.loss_type == "sigmoid":
            return self.sigmoid
    def linear_activation_function():
    def sigmoid():
    
    def set_previous_layer(self, prev_layer):
        self.prev_layer = prev_layer

    def set_next_layer(self, next_layer):
        self.next_layer = next_layer

    def initialize_weights(self, num_neurons_prev):
        # Initialize input weights matrix with random values
        self.input_weights = np.random.rand(self.N, num_neurons_prev)

        # Initialize output weights matrix with random values
        if self.next_layer:
            self.output_weights = np.random.rand(self.next_layer.N, self.N)

    def forward_pass(self, input_data):
        # Perform the forward pass through the layer
        if self.input_weights is None or (self.next_layer and self.output_weights is None):
            raise ValueError("Weights not initialized. Call initialize_weights() first.")

        # Calculate the weighted sum of inputs
        weighted_sum = np.dot(self.input_weights, input_data)

        # Apply activation function
        if self.activation_function == 'linear':
            output_data = weighted_sum
        elif self.activation_function == 'tanh':
            output_data = np.tanh(weighted_sum)
        # Add more activation functions as needed

        return output_data

    